In [4]:
import numpy as np 
import re 
from bs4 import BeautifulSoup
import spacy 
import sys
sys.path.append('scripts')
import pysbd

import json 
import re 
import random 
from spacy.tokens import DocBin
import jsonlines
from spacy.tokens import Span
from prodigy.components.db import connect
from prettytable import PrettyTable
import srsly
from prodigy.components.db import connect
from pathlib import Path 
from spacy import displacy
from pysbd.utils import PySBDFactory
from components import *

In [5]:
nlp = spacy.load("training/model-best")
nlp.pipe_names

['tok2vec',
 'Find_LatexTable',
 'pysbd_sentencizer',
 'ner',
 'custom_sentencizer']

In [6]:
text = "Expliquez comment vous déterminez les concentrations des solutions de la gamme étalon. Tout d'abord, on va déterminer les concentrations maximale et minimale de notre gamme étalon. Pour cela, on sait que notre spectrophotomètre est linéaire pour des absorbances se trouvant en 0.05 et 2. Donc grace à la formule A=C*E*l on peut déterminer les concentrations des extrêmes : § C min=A E×l=0.05104×1=5×10-6mol/L § C max=2104×1=2×10-4mol/L § Donc notre gamme étalon doit être entre 5*10-6 et 2*10-4mol/L. Ensuite, on peut maintenant déterminer les volumes maximum et minimum à prélever de solution de sel de Mohr grâce à la formule Cm*Vm=Cf*Vf § V min=Cf×Vf Cm=5×10-6×0.01 2×10-3=2.5×10-5 L=25µL § V max=2×10-4×0.01 2×10-3=1×10-3 L=1mL=1000µL § Donc on va devoir prélever des volumes de sel de Mohr entre 25 et 1000µL.Enfin, nous avons 8 tubes plastiques de 10mL donc on va faire 8solutions pour notre gamme étalon et nous voulons des valeurs à peu près espacées de la même valeurs pour avoir une courbe facile a determiner. On va donc prendre les valeurs de volumes de solution de Mohr suivantes (en µL): 25, 100, 250, 400, 550, 700, 850, 1000. Ce qui nous donne des concentrations qui leur correspondent que l'on obtient grâce a la formule Cm*Vm=Cf*Vf : § ¥¥Volume de Sel de Mohr à prélever (en µL)¥ Concentration des solutions de la gamme étalon (en mol/L¥ 25¥ 5*10-6¥ 100¥ 2*10-5¥ 250¥ 5*10-5¥ 400¥ 8*10-5¥ 550¥ 1.1*10-4¥ 700¥ 1.4*10-4¥ 850¥ 1.7*10-4¥ 1000¥ 2*10-4¥ ¥¥ "
doc = nlp(text)

In [7]:
count = 0
for sent in doc.sents:
    count += 1
    print(f"Phrase {count}: {sent}")


Phrase 1: Expliquez comment vous déterminez les concentrations des solutions de la gamme étalon.
Phrase 2: Tout d'abord, on va déterminer les concentrations maximale et minimale de notre gamme étalon.
Phrase 3: Pour cela, on sait que notre spectrophotomètre est linéaire pour des absorbances se trouvant en 0.05 et 2.
Phrase 4: Donc grace à la formule A=C*E*l on peut déterminer les concentrations des extrêmes : §
Phrase 5: C min=A E×l=0.05104×1=5×10-6mol/L §
Phrase 6: C max=2104×1=2×10-4mol/L §
Phrase 7: Donc notre gamme étalon doit être entre 5*10-6 et 2*10-4mol/L. Ensuite, on peut maintenant déterminer les volumes maximum et minimum à prélever de solution de sel de Mohr grâce à la formule Cm*Vm=Cf*Vf §
Phrase 8: V min=Cf×Vf Cm=5×10-6×0.01 2×10-3=2.5×10-5 L=25µL §
Phrase 9: V max=2×10-4×0.01 2×10-3=1×10-3 L=1mL=1000µL §
Phrase 10: Donc on va devoir prélever des volumes de sel de Mohr entre 25 et 1000µL.Enfin, nous avons 8 tubes plastiques de 10mL donc on va faire 8solutions pour notre g

/Users/anis/miniforge3/lib/python3.9/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/Users/anis/miniforge3/lib/python3.9/site-packages/spacy/displacy/__init__.py:128: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


OSError: [Errno 48] Address already in use

In [8]:
nlp.disable_pipes(["custom_sentencizer","Find_LatexTable"])
print(nlp.pipe_names)
doc = nlp(text)
count = 0
for sent in doc.sents:
    count += 1
    print(f"Phrase {count}: {sent}")


['tok2vec', 'pysbd_sentencizer', 'ner']
Phrase 1: Expliquez comment vous déterminez les concentrations des solutions de la gamme étalon. Tout d'abord, on va déterminer les concentrations maximale et minimale de notre gamme étalon. Pour cela, on sait que notre spectrophotomètre est linéaire pour des absorbances se trouvant en 0.05 et 2. Donc grace à la formule A=C*E*l on peut déterminer les concentrations des extrêmes : § C min=A E×l=0.05104×1=5×10-6mol/L § C max=2104×1=2×10-4mol/L § Donc notre gamme étalon doit être entre 5*10-6 et 2*10-4mol/L. Ensuite, on peut maintenant déterminer les volumes maximum et minimum à prélever de solution de sel de Mohr grâce à la formule Cm*Vm=Cf*Vf § V min=Cf×Vf Cm=5×10-6×0.01 2×10-3=2.5×10-5 L=25µL § V max=2×10-4×0.01 2×10-3=1×10-3 L=1mL=1000µL § Donc on va devoir prélever des volumes de sel de Mohr entre 25 et 1000µL.Enfin, nous avons 8 tubes plastiques de 10mL donc on va faire 8solutions pour notre gamme étalon et nous voulons des valeurs à peu près 